### Import Modules

In [7]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

matplotlib.rcParams['figure.figsize'] = [10, 10]

In [22]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter/special_functions.pyc'>

### Specify fit parameters

In [26]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'fit_type' : 'ma', #'ma', 'ma-taylor, 'xpt', or 'xpt-taylor'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'
    
    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],
    
    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,
    
    'save_prior' : False,
    'save_results' : False,
}

### Do fit

In [27]:
t0 = time.time()

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()

# Get prior
prior = None
if p_dict['use_prior']:
    prior = data_loader.get_prior(p_dict['fit_type'])

# Make bootstrapper
bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                               fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

if p_dict['make_plots']:
    data_loader.save_plots(
        bootstrapper.make_plots(
            show_error_ellipses=p_dict['show_error_ellipses'], 
            show_bootstrap_histograms=p_dict['show_bs_histograms']),
        output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
    )
else:
    print bootstrapper
    
if False: #p_dict['save_prior']:
    print 'Saving prior...'
    data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])
    
if p_dict['save_results']:
    data_loader.save_fit_info(bootstrapper.get_fit_info())
    
    
t1 = time.time()

print "\nTotal time (s): ", t1 - t0, "\n"

Using default prior.
Making fits...
100% complete Time (s):  2.21654176712
Compiling results...
Done.

Total time (s):  6.35343599319 



In [66]:
def test(self):
    temp_prior  = {}
    for key in self.get_fit_parameters().keys():
        #temp_prior[key] = gv.gvar(0, 0)
        if True: #key in ['L_5']:
            value = np.random.normal(gv.mean(self.get_fit_parameters(key)), gv.sdev(self.get_fit_parameters(key)))
            temp_prior[key] = value
        
    return self.fk_fpi_fit_fcn(fit_parameters=temp_prior)

test(bootstrapper)

0.9682(30)

In [76]:
myarray = [test(bootstrapper) for j in range(100)]
np.median(myarray)

1.1745(47)

In [116]:
def test(self):
    temp_data = self._make_fit_data(0)
    eps2_a = np.median((temp_data['a/w0'] / (4 *np.pi))**2)
    eps2_k = np.median((temp_data['mk'])**2 / temp_data['lam2_chi'])
    eps2_pi = np.median((temp_data['mpi'])**2 / temp_data['lam2_chi'])
    temp = self.fits[0].fcn_p
    fk_fpi = (np.median(temp[temp.keys()[0]]))
    
    for key in self.get_fit_parameters():
        value = 0
        if key == 'A_a':
            value = self.get_fit_parameters(key) *eps2_a
            
        elif key == 'A_k':
            value = self.get_fit_parameters(key) *eps2_k
            
        elif key == 'A_p':
            value = self.get_fit_parameters(key) *eps2_pi
        
        elif key == 'A_aa':
            value = self.get_fit_parameters(key) *eps2_a**2
            
        elif key == 'A_ak':
            value = self.get_fit_parameters(key) *eps2_a *eps2_k
            
        elif key == 'A_ap':
            value = self.get_fit_parameters(key) *eps2_a *eps2_pi
            
        elif key == 'A_kk':
            value = self.get_fit_parameters(key) *eps2_k**2
            
        elif key == 'A_kp':
            value = self.get_fit_parameters(key) *eps2_k *eps2_pi
            
        elif key == 'A_pp':
            value = self.get_fit_parameters(key) *eps2_pi**2
        
        value = value *(eps2_k - eps2_pi)
        if key not in ['L_4', 'L_5']:
            if ((np.abs(gv.mean(value)+gv.sdev(value)) < gv.sdev(fk_fpi)) 
                    and (np.abs(gv.mean(value)-gv.sdev(value)) < gv.sdev(fk_fpi))):
                print key, "is hopeless:", value, 'vs', gv.sdev(fk_fpi)
    
test(bootstrapper)

A_ap is hopeless: -0.00001(18) vs 0.00106834119412
A_ak is hopeless: -0.00004(53) vs 0.00106834119412
A_aa is hopeless: -2(1035)e-08 vs 0.00106834119412


### Or make all fits

In [30]:
p_dict = {
    'order' : {
        'vol' : 8 # max 10
    }, 
    'bs_N' : 1,  # if 0, use full list
    'F2' : 'FKFpi', #'FKfpi', 'FpiFpi', 'FKFK'

    'use_prior' : False,
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m310', u'a12m350',
           u'a12m400', u'a15m220', u'a15m310', u'a15m350', 'a15m400'],

    'make_plots' : True,
    'show_error_ellipses' : False,
    'show_bs_histograms' : False,

    'save_prior' : False,
    'save_results' : True,
}

In [31]:
for fit_type in ['xpt', 'xpt-taylor', 'ma', 'ma-taylor', 'ma-old']:
    for order_fit in ['nlo', 'nnlo', 'nnnlo']:
        
        print fit_type, order_fit
        
        # Specify fit parameters
        p_dict['order']['fit'] = order_fit
        p_dict['fit_type'] = fit_type
        
        # Do fit
        t0 = time.time()

        # Load data
        data_loader = dl.data_loader()
        fit_data = data_loader.get_fit_data()

        # Get prior
        prior = None
        if p_dict['use_prior']:
            prior = data_loader.get_prior(p_dict['fit_type'])

        # Make bootstrapper
        bootstrapper = bs.bootstrapper(fit_data, prior=prior, order=p_dict['order'], F2=p_dict['F2'],
                                       fit_type=p_dict['fit_type'], bs_N=p_dict['bs_N'], abbrs=p_dict['abbrs'])

        if p_dict['make_plots']:
            data_loader.save_plots(
                bootstrapper.make_plots(
                    show_error_ellipses=p_dict['show_error_ellipses'], 
                    show_bootstrap_histograms=p_dict['show_bs_histograms']),
                output_filename=bootstrapper.fit_type+'_'+bootstrapper.order['fit']
            )
        else:
            print bootstrapper

        if p_dict['save_prior']:
            print 'Saving prior...'
            data_loader.save_prior(bootstrapper.create_prior_from_fit(), p_dict['fit_type'])

        if p_dict['save_results']:
            data_loader.save_fit_info(bootstrapper.get_fit_info())


        t1 = time.time()
        
        print test(bootstrapper)

        print "\nTotal time (s): ", t1 - t0, "\n"

xpt nlo
Using default prior.
Making fits...
100% complete Time (s):  1.42708206177
Compiling results...
Done.
Saving...
Done.
1.1335(13)

Total time (s):  4.09499597549 

xpt nnlo
Using default prior.
Making fits...
100% complete Time (s):  1.88571190834
Compiling results...
Done.
Saving...
Done.
1.1335(13)

Total time (s):  4.6727180481 

xpt nnnlo
Using default prior.
Making fits...
100% complete Time (s):  1.98129487038
Compiling results...
Done.
Saving...
Done.
1.1335(13)

Total time (s):  4.87654495239 

xpt-taylor nlo
Using default prior.
Making fits...
100% complete Time (s):  2.09442901611
Compiling results...
Done.
Saving...
Done.
1.1598(21)

Total time (s):  5.65817093849 

xpt-taylor nnlo
Using default prior.
Making fits...
100% complete Time (s):  1.99474477768
Compiling results...
Done.
Saving...
Done.
1.1598(21)

Total time (s):  5.16720604897 

xpt-taylor nnnlo
Using default prior.
Making fits...
100% complete Time (s):  2.02389597893
Compiling results...
Done.
Saving...

In [38]:
#bootstrapper.fits[0].show_plots(view='std')

In [33]:
a = gv.gvar(0.8, 4)
b = gv.gvar(0.5, 1)
c = gv.gvar(-3.4, 7.2)

a*b*c

-1.4(7.9)